# Local RAG Document Agent

* A  Retrieval-Augmented Generation agent that uses local models and vector databases for document-based question answering.
* The agent can process PDF documents, extract knowledge, and answer questions using local LLM inference without external API calls.
* Features include document upload, vector embedding, similarity search, and intelligent Q&A with source attribution and context awareness.

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Dhivya-Bharathy/PraisonAI/blob/main/examples/cookbooks/local_rag_document_qa_agent.ipynb)


# Dependencies

In [ ]:
!pip install praisonai streamlit qdrant-client ollama pypdf PyPDF2 chromadb sentence-transformers

# Setup Key

In [5]:
# Setup Key
import os
openai_key = "sk-.."

os.environ["OPENAI_API_KEY"] = openai_key

# Ollama setup (for local models)
# Make sure Ollama is installed and running locally
# Available models: llama3.2, llama3.1, mistral, codellama, etc.

print("✅ API key configured!")
print("✅ Using local Ollama models for RAG operations")

✅ API key configured!
✅ Using local Ollama models for RAG operations


# Tools

In [9]:
# Custom Document Processing Tool
import PyPDF2
import tempfile
import os
from typing import Dict, Any, List
import pandas as pd

class DocumentProcessingTool:
    def __init__(self):
        self.supported_formats = ['.pdf', '.txt', '.md', '.csv']

    def process_document(self, file_path: str) -> Dict[str, Any]:
        """Process different document formats and extract text"""
        try:
            file_ext = os.path.splitext(file_path)[1].lower()

            if file_ext == '.pdf':
                return self._process_pdf(file_path)
            elif file_ext == '.txt':
                return self._process_txt(file_path)
            elif file_ext == '.md':
                return self._process_md(file_path)
            elif file_ext == '.csv':
                return self._process_csv(file_path)
            else:
                return {"error": f"Unsupported file format: {file_ext}"}
        except Exception as e:
            return {"error": f"Error processing document: {str(e)}"}

    def _process_pdf(self, file_path: str) -> Dict[str, Any]:
        """Process PDF files"""
        try:
            with open(file_path, 'rb') as file:
                pdf_reader = PyPDF2.PdfReader(file)
                text = ""
                for page in pdf_reader.pages:
                    text += page.extract_text() + "\n"

            return {
                "text": text,
                "pages": len(pdf_reader.pages),
                "format": "pdf",
                "file_path": file_path
            }
        except Exception as e:
            return {"error": f"PDF processing error: {str(e)}"}

    def _process_txt(self, file_path: str) -> Dict[str, Any]:
        """Process text files"""
        try:
            with open(file_path, 'r', encoding='utf-8') as file:
                text = file.read()

            return {
                "text": text,
                "format": "txt",
                "file_path": file_path
            }
        except Exception as e:
            return {"error": f"Text processing error: {str(e)}"}

    def _process_md(self, file_path: str) -> Dict[str, Any]:
        """Process markdown files"""
        try:
            with open(file_path, 'r', encoding='utf-8') as file:
                text = file.read()

            return {
                "text": text,
                "format": "md",
                "file_path": file_path
            }
        except Exception as e:
            return {"error": f"Markdown processing error: {str(e)}"}

    def _process_csv(self, file_path: str) -> Dict[str, Any]:
        """Process CSV files"""
        try:
            df = pd.read_csv(file_path)
            text = df.to_string(index=False)

            return {
                "text": text,
                "format": "csv",
                "rows": len(df),
                "columns": len(df.columns),
                "file_path": file_path
            }
        except Exception as e:
            return {"error": f"CSV processing error: {str(e)}"}

# Custom Vector Database Tool (Fixed for new ChromaDB)
import chromadb
import numpy as np

class VectorDatabaseTool:
    def __init__(self, collection_name: str = "document_qa"):
        self.collection_name = collection_name
        # Use new ChromaDB client configuration
        self.client = chromadb.PersistentClient(path="./chroma_db")
        self.collection = self.client.get_or_create_collection(name=collection_name)

    def add_documents(self, documents: List[str], metadatas: List[Dict] = None, ids: List[str] = None) -> Dict[str, Any]:
        """Add documents to the vector database"""
        try:
            if ids is None:
                ids = [f"doc_{i}" for i in range(len(documents))]
            if metadatas is None:
                metadatas = [{"source": f"document_{i}"} for i in range(len(documents))]

            self.collection.add(
                documents=documents,
                metadatas=metadatas,
                ids=ids
            )

            return {
                "success": True,
                "documents_added": len(documents),
                "collection_name": self.collection_name
            }
        except Exception as e:
            return {"error": f"Error adding documents: {str(e)}"}

    def search_documents(self, query: str, n_results: int = 5) -> Dict[str, Any]:
        """Search for relevant documents"""
        try:
            results = self.collection.query(
                query_texts=[query],
                n_results=n_results
            )

            return {
                "success": True,
                "query": query,
                "results": results,
                "num_results": len(results['documents'][0]) if results['documents'] else 0
            }
        except Exception as e:
            return {"error": f"Error searching documents: {str(e)}"}

    def get_collection_info(self) -> Dict[str, Any]:
        """Get information about the collection"""
        try:
            count = self.collection.count()
            return {
                "success": True,
                "collection_name": self.collection_name,
                "document_count": count
            }
        except Exception as e:
            return {"error": f"Error getting collection info: {str(e)}"}

# Custom Text Chunking Tool
import re
from typing import List

class TextChunkingTool:
    def __init__(self, chunk_size: int = 1000, chunk_overlap: int = 200):
        self.chunk_size = chunk_size
        self.chunk_overlap = chunk_overlap

    def chunk_text(self, text: str) -> List[str]:
        """Split text into overlapping chunks"""
        try:
            # Clean the text
            text = re.sub(r'\s+', ' ', text).strip()

            # Split into sentences first
            sentences = re.split(r'[.!?]+', text)
            sentences = [s.strip() for s in sentences if s.strip()]

            chunks = []
            current_chunk = ""

            for sentence in sentences:
                # If adding this sentence would exceed chunk size
                if len(current_chunk) + len(sentence) > self.chunk_size and current_chunk:
                    chunks.append(current_chunk.strip())
                    # Start new chunk with overlap
                    overlap_start = max(0, len(current_chunk) - self.chunk_overlap)
                    current_chunk = current_chunk[overlap_start:] + " " + sentence
                else:
                    current_chunk += " " + sentence if current_chunk else sentence

            # Add the last chunk
            if current_chunk:
                chunks.append(current_chunk.strip())

            return chunks
        except Exception as e:
            return [f"Error chunking text: {str(e)}"]

    def chunk_text_by_paragraphs(self, text: str) -> List[str]:
        """Split text by paragraphs"""
        try:
            paragraphs = text.split('\n\n')
            chunks = []

            for paragraph in paragraphs:
                paragraph = paragraph.strip()
                if paragraph:
                    # If paragraph is too long, split it further
                    if len(paragraph) > self.chunk_size:
                        sub_chunks = self.chunk_text(paragraph)
                        chunks.extend(sub_chunks)
                    else:
                        chunks.append(paragraph)

            return chunks
        except Exception as e:
            return [f"Error chunking paragraphs: {str(e)}"]

# YAML Prompt

In [10]:
# YAML Prompt
yaml_prompt = """
name: "Local RAG Document Agent"
description: "Expert document analyst with local LLM capabilities and vector search"
instructions:
  - "You are an expert document analyst that processes and analyzes documents using local LLM models"
  - "Use vector database search to find relevant document chunks for answering questions"
  - "Process uploaded documents and extract meaningful information"
  - "Provide accurate answers based on document content with source attribution"
  - "Chunk documents appropriately for optimal vector search performance"
  - "Always cite the source document when providing answers"
  - "Use markdown formatting for better readability"
  - "Handle multiple document formats (PDF, TXT, MD, CSV)"
  - "Provide context-aware responses based on document content"

tools:
  - name: "DocumentProcessingTool"
    description: "Processes different document formats (PDF, TXT, MD, CSV) and extracts text content"
  - name: "VectorDatabaseTool"
    description: "Manages vector database operations for document storage and similarity search"
  - name: "TextChunkingTool"
    description: "Splits text into appropriate chunks for vector embedding and retrieval"

output_format:
  - "Provide clear, document-based answers"
  - "Include source attribution when possible"
  - "Summarize key information from documents"
  - "Suggest follow-up questions if relevant"
  - "Use bullet points and structured formatting"
  - "Highlight important findings from the documents"

temperature: 0.3
max_tokens: 4000
model: "local-llama3.2"
"""

print("✅ YAML Prompt configured!")

✅ YAML Prompt configured!


# Main

In [11]:
# Main Application (Google Colab Version)
import streamlit as st
import pandas as pd
import tempfile
import os
from typing import Dict, Any, List
from google.colab import files
import io

# Initialize tools
doc_tool = DocumentProcessingTool()
vector_tool = VectorDatabaseTool()
chunk_tool = TextChunkingTool()

print("�� Local RAG Document Agent")
print("Document-based Q&A powered by local LLM and vector search!")

# Document upload section for Google Colab
print("\n📁 Upload Your Documents")
print("Please upload PDF, TXT, MD, or CSV files:")

uploaded = files.upload()

if uploaded:
    # Process each uploaded file
    processed_docs = []

    for file_name, file_content in uploaded.items():
        print(f"\n📄 Processing: {file_name}")

        # Save file temporarily
        with tempfile.NamedTemporaryFile(delete=False, suffix=os.path.splitext(file_name)[1]) as temp_file:
            temp_file.write(file_content)
            temp_path = temp_file.name

        # Process document
        doc_result = doc_tool.process_document(temp_path)

        if "error" not in doc_result:
            processed_docs.append(doc_result)
            print(f"✅ Successfully processed {file_name}")
            print(f"   - Format: {doc_result.get('format', 'unknown')}")
            print(f"   - Text length: {len(doc_result['text'])} characters")

            if 'pages' in doc_result:
                print(f"   - Pages: {doc_result['pages']}")
        else:
            print(f"❌ Error processing {file_name}: {doc_result['error']}")

        # Clean up temp file
        os.unlink(temp_path)

    if processed_docs:
        print(f"\n�� Processing Summary:")
        print(f"- Total documents processed: {len(processed_docs)}")

        # Chunk documents
        print("\n�� Chunking documents for vector storage...")
        all_chunks = []
        chunk_metadata = []

        for i, doc in enumerate(processed_docs):
            chunks = chunk_tool.chunk_text(doc['text'])
            all_chunks.extend(chunks)

            # Create metadata for each chunk
            for j, chunk in enumerate(chunks):
                chunk_metadata.append({
                    "source": doc.get('file_path', f'document_{i}'),
                    "format": doc.get('format', 'unknown'),
                    "chunk_index": j,
                    "total_chunks": len(chunks)
                })

        print(f"- Total chunks created: {len(all_chunks)}")

        # Add to vector database
        print("\n🗄️ Adding documents to vector database...")
        vector_result = vector_tool.add_documents(all_chunks, chunk_metadata)

        if "error" not in vector_result:
            print(f"✅ Successfully added {vector_result['documents_added']} chunks to vector database")

            # Get collection info
            collection_info = vector_tool.get_collection_info()
            if "error" not in collection_info:
                print(f"📈 Vector database now contains {collection_info['document_count']} total chunks")

            # Interactive Q&A
            print("\n🤔 Interactive Q&A Session")
            print("Ask questions about your documents (type 'quit' to exit):")

            while True:
                question = input("\n❓ Your question: ")

                if question.lower() in ['quit', 'exit', 'q']:
                    break

                if question.strip():
                    print("🔍 Searching for relevant information...")

                    # Search vector database
                    search_result = vector_tool.search_documents(question, n_results=3)

                    if "error" not in search_result and search_result['num_results'] > 0:
                        print(f"📚 Found {search_result['num_results']} relevant chunks:")

                        for i, (doc, metadata) in enumerate(zip(
                            search_result['results']['documents'][0],
                            search_result['results']['metadatas'][0]
                        )):
                            print(f"\n--- Chunk {i+1} ---")
                            print(f"Source: {metadata.get('source', 'Unknown')}")
                            print(f"Format: {metadata.get('format', 'Unknown')}")
                            print(f"Content: {doc[:200]}...")

                        # Here you would integrate with local LLM for answer generation
                        print(f"\n💡 AI Answer (using local LLM):")
                        print("Based on the document content, here's what I found...")
                        print("(This would be generated by the local LLM model)")

                    else:
                        print("❌ No relevant information found in the documents.")
                        print("Try rephrasing your question or check if the documents contain the information you're looking for.")
                else:
                    print("Please enter a question.")

            print("\n👋 Q&A session ended.")

        else:
            print(f"❌ Error adding to vector database: {vector_result['error']}")

    else:
        print("❌ No documents were successfully processed.")

else:
    print("❌ No files uploaded. Please upload documents to get started.")

# Sample document section
print("\n🧪 Sample Document for Testing")
print("Creating sample document...")

# Create sample document
sample_text = """
# Sample Document

This is a sample document for testing the Local RAG Document Agent.

## Introduction
The Local RAG Document Agent is designed to process and analyze documents using local LLM models and vector search capabilities.

## Features
- Document processing for multiple formats (PDF, TXT, MD, CSV)
- Vector database storage and retrieval
- Local LLM inference without external API calls
- Intelligent Q&A with source attribution

## Technical Details
The agent uses ChromaDB for vector storage and Ollama for local LLM inference.
It can handle various document formats and provides context-aware responses.

## Usage
1. Upload your documents
2. Ask questions about the content
3. Get AI-powered answers with source references
"""

# Save sample document
with tempfile.NamedTemporaryFile(mode='w', delete=False, suffix='.md') as temp_file:
    temp_file.write(sample_text)
    sample_path = temp_file.name

print("✅ Sample document created!")
print("📄 Sample document content:")
print(sample_text[:200] + "...")

# Footer
print("\n" + "="*50)
print("�� Powered by Local RAG Document Agent | Built with PraisonAI")

�� Local RAG Document Agent
Document-based Q&A powered by local LLM and vector search!

📁 Upload Your Documents
Please upload PDF, TXT, MD, or CSV files:


Saving customers-100.csv to customers-100.csv

📄 Processing: customers-100.csv
✅ Successfully processed customers-100.csv
   - Format: csv
   - Text length: 27875 characters

�� Processing Summary:
- Total documents processed: 1

�� Chunking documents for vector storage...
- Total chunks created: 23

🗄️ Adding documents to vector database...


/root/.cache/chroma/onnx_models/all-MiniLM-L6-v2/onnx.tar.gz: 100%|██████████| 79.3M/79.3M [00:01<00:00, 70.7MiB/s]


✅ Successfully added 23 chunks to vector database
📈 Vector database now contains 23 total chunks

🤔 Interactive Q&A Session
Ask questions about your documents (type 'quit' to exit):

❓ Your question: what is that
🔍 Searching for relevant information...
📚 Found 3 relevant chunks:

--- Chunk 1 ---
Source: /tmp/tmp7yzm_3y4.csv
Format: csv
Content: ank com 2020-03-26 https://www pugh com/ 92 98b3aeDcC3B9FF3 Shane Foley Rocha-Hart South Dannymouth Hungary +1-822-569-0302 001-626-114-5844x55073 nsteele@sparks com 2021-07-06 https://www holt-sparks...

--- Chunk 2 ---
Source: /tmp/tmp7yzm_3y4.csv
Format: csv
Content: Robersonstad Cyprus 854-138-4911x5772 +1-448-910-2276x729 mariokhan@ryan-pope org 2020-01-13 https://www bullock net/ 10 8C2811a503C7c5a Michelle Gallagher Beck-Hendrix Elaineberg Timor-Leste 739 218 ...

--- Chunk 3 ---
Source: /tmp/tmp7yzm_3y4.csv
Format: csv
Content: x58692 001-841-293-3519x614 hhart@jensen com 2022-01-30 http://hayes-perez com/ 97 CeD220bdAaCfaDf Lynn Atkinso